# Calculating Headways for CTA Buses - work in progress

NOTE: Headways for late night routes are not accurate yet - See the Needs Fixing section below.

### What This Does

This code calculates headways for each CTA bus stop on a given route and direction.  It also generates
summary stats for an entire route (min, max, mean, median, 25th percentile, and 75th percentile headways).

### To Do Next

- Fix late night headway issue

- Add summary stats to the stops geodataframe so they show up in the visualizations when you hover over a stop

- See what happens if you try to loop through many - or all - routes and compile a larger headways dataframe

- Investigate EWT calcs Sean MacMullan found: https://www.trapezegroup.com.au/resources/infographic-how-to-calculate-excess-waiting-time/ 

### Notes

One bus route can be made up of several patterns.  Headways are calculated for all buses running the same direction on a given route at a particular stop, regardless which pattern the bus is on.   

Vehicle data comes from the Chi Hack Night Ghost Buses breakout team: https://ghostbuses.com/about
This provides location information in 5 minute intervals for every CTA bus.  It includes
data on which route (rt) and pattern (pid) the bus was running, along with the vehicle's distance along the pattern (pdist) and a timestamp.  

Pattern data comes from the CTA's API directly. This tells us which stops are found along
a given pattern and the distance along the pattern where each stop is located.

Combining the datasets above, the general strategy is:

1. Turn vehicle data into intervals:  Time and distance are recorded at the start and end of each 5-minute interval.

2. For a given stop and pattern, find all intervals where a vehicle on that pattern reached or pased the stop.

3. Estimate the time each bus acutally reached the stop through interpolation.  The interval gives time and distance location along a given pattern before and after the bus arrived at the stop.  The CTA's pattern data tells us where the bus stop falls along the pattern.  Stop times are estimated assuming the vehicle travels a constant spaeed througout the interval.

4. Combine all stop times for buses running the same direction at a particular stop.

5. Calculate headways between buses based on stop times.

6. Calculate summary statistics on headways.


### Needs Fixing

Address late night routes. 

- Some buses run past midnight, so the first bus of the calendar day appears just after midnight. 
- Many of these have no scheduled service in the early morning hours, but the unscheduled hours show up as a long headway.  And different directions of travel may begin and end service at different times. 
    - Need to change the range of times where data is used so it doesn't assume buses should be running during their off time  
    - (Example: see first bus/last bus info on the 55 bus here: https://www.transitchicago.com/bus/55/)
        - Parts of this route run 24 hours
        - Parts run 5am to 2am
        - Parts run 4:10 am to 1:10am
        - First and last buses reach mid-route stops later than these ranges. Buses continue to the end of their route
        - not in service times between 5am and 2am show up in the code below as long headways
 
- Possible Approach?

    - Combine 2 days of data: the "main" date being looked at and the following date for any schedules that splill into overnight.  How to choose a new range that starts and ends with the out-of-service period?

    - Investigate the schedule data from CTA. Can we extract actual start/end times for a route/direction, adding a buffer of extra time on the end to capture buses finishing their routes at end of the schedule? 

- Also, this code currently eliminates all intervals where one timestamp is before midnight and the next is after, since it uses one calendar day of data.  Missing one inverval per day is less critical than the issue above, and it would likely be resolved by the same fixes.




In [4]:
import requests
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from shapely import Point, LineString
import datetime as dt
import numpy as np

In [5]:
# Get API key from the .env file
load_dotenv()
API_KEY = os.getenv('API_KEY')

### Functions

In [18]:
def get_chn_vehicles(date_string:str, start_timedelta_string:str='02:30', end_timedelta_string:str='26:30') -> pd.DataFrame:
    """Parameters:\n

    date_string in 'YYYY-MM-DD'format\n

    start_timedelta_string in 'hh:mm' format.  (optional. Default is '02:30')\n
    end_timedelta_string in 'hh:mm' format. (optional. Default is '26:30)' \n

    Data returned:\n

    Vehicle data scraped by the chn ghost bus team for all CTA buses running between the specified
    start and end times on the specified date.  Where an end time over 24 hours is specified, the data returned
    will extend into the following calendar date. The maximum valid 
    value for start_timedelta_string or end_timedelta_string is '23:59'\n
    
    Timedelta values start at midnight on day 1.  Example: If start_timedelta_string is '03:45' 
    and end_timedelta_sring is '25:07', then data returned is from 3:45 am on the requested date 
    through 1:07 am the following day. \n
    
    Data is returned in a pandas dataframe. Columns include vehicle id (vid), timestamp (tmstmp), 
    pattern id (pid), and distance along the pattern (pdist) for each vehicle at 5-minute intervals 
    throughout the requested time range on the requested calendar day.
    
    """

    day1 = pd.to_datetime(date_string, infer_datetime_format=True)
    day2 = day1 + pd.Timedelta(days=1)
    day2_string = day2.strftime('%Y-%m-%d')

    start_timedelta_string_expanded = start_timedelta_string + ':00'
    end_timedelta_string_expanded = end_timedelta_string + ':00'

    def get_vehicles_single_day(single_day_datestring):
        chn_data_source_single_day = f'https://chn-ghost-buses-public.s3.us-east-2.amazonaws.com/bus_full_day_data_v2/{single_day_datestring}.csv'
        vehicles_single_day = pd.read_csv(
        chn_data_source_single_day, dtype={
            'vid':'int',
            'tmstmp':'str',
            'lat':'float',
            'lon':'float',
            'hdg':'int',
            'pid':'int',
            'rt':'str',
            'pdist':'int',
            'des':'str',
            'dly':'bool',
            'tatripid':'str',
            'origatripno':'int',
            'tablockid':'str',
            'zone':'str',
            'scrape_file':'str',
            'data_hour':'int',
            'data_date':'str'
            }
        )

        vehicles_single_day['tmstmp'] = pd.to_datetime(vehicles_single_day['tmstmp'],infer_datetime_format=True)
    
        return vehicles_single_day

    df_day1_vehicles = get_vehicles_single_day(date_string)
    df_day2_vehicles = get_vehicles_single_day(day2_string)
    
    df_both_days_vehicles = pd.concat([df_day1_vehicles, df_day2_vehicles])

    # Filter for vehicles running between 2:50 am on day 1 and 2:50 am on day 2.
    # Note:  First tried 2:30, but caught one bus on the test day / route that had their
    # out of service time based on actual stop times from 3:42 am to 4:37 am.  
    # Then changed the window to 3:50am and found another bus stop with a bus
    # at 4:11 and another at 5:50.
    # another 

    # TODO
    # Possible to run quickly check every bus route and see if the longest headway is actually
    # an out of service time around 3am?

    service_day_start = day1+pd.Timedelta(start_timedelta_string_expanded)
    service_day_end = day1+pd.Timedelta(end_timedelta_string_expanded)

    df_vehicles = df_both_days_vehicles.loc[
        (service_day_start < df_both_days_vehicles['tmstmp'])
        & (df_both_days_vehicles['tmstmp'] <= service_day_end)]
 
    return df_vehicles


In [ ]:

    # Filter for vehicles running between 2:50 am on day 1 and 2:50 am on day 2.
    # Note:  First tried 2:30, but caught one bus on the test day / route that had their
    # out of service time based on actual stop times from 3:42 am to 4:37 am.  
    # Then changed the window to 3:50am and found another bus stop with a last bus
    # at 4:11 and a first bus at 5:50.
    # another 

    # TODO
    # Possible to run quickly check every bus route and see if the longest headway is actually
    # an out of service time around 3am?

    service_day_start = day1+pd.Timedelta(hours=2, minutes=50)
    service_day_end = day1+pd.Timedelta(hours=26, minutes=50)

    df_vehicles = df_both_days_vehicles.loc[
        (service_day_start < df_both_days_vehicles['tmstmp'])
        & (df_both_days_vehicles['tmstmp'] <= service_day_end)]
 
    return df_vehicles


In [ ]:
def get_patterns(vehicles:pd.DataFrame, rt:str) -> pd.DataFrame:
    '''This is a helper function.\n
    Parameters:\n
    vehicles is a dataframe obtained using get_chn_vehicles().\n
    rt is a route id as a string (for example, '55' for the 55 Garfield bus)\n
    Data returned:\n
    patterns data from the CTA's bus tracker API is returned in a dataframe. 
    It includes all pattern ids (pid) found in the in the vehicles data for the specified
    route. Columns include pattern id (pid) and points (pt).\n
    The pt data for each pattern is its own dataframe with information on every point
    along the pattern. It includes columns for sequence (seq), latitude (lat),
    longitude (lon), type of points (typ) where S indicates a bus stop,
    stop ID (stpid) for stop points, and distance along the pattern (pdist).
     '''
    

    df_output = pd.DataFrame()

    # filter vehicles to the specified route
    rt_vehicles = vehicles.loc[vehicles['rt'] == rt]

    # list pid values included in the route
    pid_list = list(rt_vehicles['pid'].unique())

    # convert pids to strings
    pid_list = [str(i) for i in pid_list]

    # split pid_list into chunks of 10 (limit of the API):
    start = 0
    end = len(pid_list)
    step = 10
    for i in range(start, end, step):
        pid_list_chunk = pid_list[i:i+step]
        pid_string = ','.join(pid_list_chunk)

        # get data from CTA's feed
        api_url = f'http://www.ctabustracker.com/bustime/api/v2/getpatterns?key={API_KEY}&pid={pid_string}&format=json'
        response = requests.get(api_url)
        patterns = response.json()

        # convert json to dataframe
        df_patterns = pd.DataFrame(patterns['bustime-response']['ptr'])

        # add to the output dataframe
        df_output = pd.concat([df_output, df_patterns])


    # convert pt column values to dataframes for each pattern containing that pattern's points
    df_output['pt'] = df_output['pt'].apply(lambda x: pd.DataFrame(x))
    
    return df_output


In [ ]:

def get_pattern_linestrings(patterns:pd.DataFrame) -> gpd.GeoDataFrame:
    '''This is for future use and visualization - not neccessary to generate
    headway information.\n
    Paremeters:\n
    patterns is a dataframe obtained using get_patterns().\n
    Data returned:\n
    Pattern data is returned as a geodataframe wiht linestring geometry
    representing the path buses travel.'''

    df_patterns = patterns.copy()

    # Turn points into linestrings
    geometry_linestrings = []
    for p in df_patterns['pt']:
        p.sort_values('seq', inplace=True)
        linestring_points = list(zip(p['lon'],p['lat']))

        # generate linestring using all points
        linestring = LineString(linestring_points)
        geometry_linestrings.append(linestring)

    # Create a geodataframe for the patterns using the linestring geometry
    gdf_patterns = gpd.GeoDataFrame(df_patterns, geometry=geometry_linestrings).set_crs(epsg=4326)

    # Drop the original pt column
    gdf_patterns.drop(['pt'], axis=1, inplace=True)

    return gdf_patterns


In [ ]:
def get_pattern_stops(patterns) -> gpd.GeoDataFrame:
        '''This is a helper function.\n
        Parameters:\n
        patterns is a dataframe obtained using get_patterns().\n
        Data returned:\n
        Bus stop data is returned as a geodataframe 
        with point geomtry, one point per bus stop on each pattern
        associated with a route.\n
        Note that stops serving multiple patterns will be listed multiple 
        times, once for each pattern with the seq and pdist values 
        specific to that pattern.'''

        # get patterns for the route
        df_patterns = patterns.copy()

        # set up a geodataframe to contain stops
        gdf_route_stops = gpd.GeoDataFrame()

        # consider the pid column (pattern ID) and the pt column (dataframe contaning
        # points along the pattern)
        for pid, pt in zip(df_patterns['pid'],df_patterns['pt']):
                # sort points sequentially
                pt.sort_values('seq', inplace=True)
                # add the pattern id to each point's data
                pt['pid']=pid
                # add the pattern direction to each point's data
                rtdir = df_patterns['rtdir'].loc[df_patterns['pid'] == pid].tolist()[0]
                pt['rtdir'] = rtdir
                # filter to only show stop points
                stops = pt[pt['typ']=='S']
                # zip lat/lon data to get coordinate pairs
                coords = list(zip(stops['lon'],stops['lat']))
                # turn coordinates into point geometry
                geometry = [Point(c) for c in coords]
                # generate a geodataframe for the stops in this pattern
                gdf_pattern_stops = gpd.GeoDataFrame(stops,geometry=geometry).set_crs(epsg=4326)
                # add this pattern's stops to the dataframe containing all stops on the route
                gdf_route_stops = pd.concat([gdf_route_stops, gdf_pattern_stops])

        return gdf_route_stops


In [ ]:
def get_vehicle_intervals(vehicles:pd.DataFrame, rt:str) -> pd.DataFrame:

    '''This is a helper function.\n
    Parameters:\n
    vehicles is a dataframe obtained using get_chn_vehicles().\n
    Data returned:\n
    Intervals are returned as a dataframe, with each row representing
    an interval between two points in time and space for one vehicle. 
    Columns are added to the vehicles data for each interval's 
    start time, end time, start pdist, and end pdist.'''

    df_vehicles = vehicles.copy()

    # filter to the specified route
    df_vehicles = df_vehicles.loc[df_vehicles['rt'] == rt]

    # Set up dataframe to contain final fomratted data
    df_output = pd.DataFrame()

    # End time for each interval as a timestamp
    # df_vehicles['end_time'] = pd.to_datetime(df_vehicles['tmstmp'],infer_datetime_format=True)
    df_vehicles['end_time'] = df_vehicles['tmstmp']
    vid_list = df_vehicles['vid'].unique().tolist()

    # End location for each interval
    df_vehicles['end_pdist'] = df_vehicles['pdist']

    for vid in vid_list:

        # pare data down to a single vehicle
        df_vehicle = df_vehicles.loc[df_vehicles['vid'] == vid]

        # handle each pattern separately
        pid_list = df_vehicle['pid'].unique().tolist()
        for p in pid_list:
            df_vehicle_pattern = df_vehicle.loc[df_vehicle['pid']==p].copy()
            # sort by time (it should be sorted already, but just in case)
            df_vehicle_pattern.sort_values(by=['end_time'], inplace=True)

            # Create a start time based on the previous tinmestamp
            end_times = df_vehicle_pattern['end_time'].tolist()
            start_times = np.roll(end_times,shift=1)
            df_vehicle_pattern['start_time'] = start_times

            # Create a start pattern distance based on the previous pdist
            end_distances = df_vehicle_pattern['end_pdist'].tolist()
            start_distances = np.roll(end_distances,shift=1)
            df_vehicle_pattern['start_pdist'] = start_distances

            # Remove the first interval since we don't have real start
            # time or location data for it
            df_vehicle_pattern = df_vehicle_pattern.iloc[1:]

            # add data to the full output dataframe
            df_output = pd.concat([df_output, df_vehicle_pattern])

    return df_output



In [ ]:
        # Interpolate estimated times the bus arrived at a stop
        def interpolate_stop_time(
            stop_pdist:int, 
            start_time:pd.Timestamp, 
            end_time:pd.Timestamp, 
            start_pdist:int, 
            end_pdist:int
            ) -> pd.Timestamp:

            '''This is a helper function.\n
            Parameters:\n
            stop_pdist is an integer distance along a pattern to a given bus stop.\n
            start_time and end_time are timestamps for the beginning and end of an interval.\n
            start_pdist and end_pdist are integer distances along a pattern at the beginning and
            end of an interval.\n
            Data returned:\n
            timestamp for the estimated time a vehicle reached a stop, assuming it
            traveled a constant speed from start to end of teh interval
            '''

            # How far into the interval distance is the bus stop?
            # stop distance from beginning of interval / full interval distance
            dist_ratio = (stop_pdist-start_pdist)/(end_pdist-start_pdist)

            # estimated bus stop time, assuming it traveled at a steady
            # speed throughout the interval
            est_stop_time = start_time + (end_time - start_time)*dist_ratio

            # round estimated stop time to the nearest minute
            est_stop_time = est_stop_time.round(freq='T')

            return est_stop_time

In [ ]:
def get_stoptimes(rt:str, vehicles:pd.DataFrame) -> pd.DataFrame:

    '''This is a helper function.\n
    Parameters:\n
    vehicles is a dataframe obtained using get_chn_vehicles().\n
    rt is a route id as a string (for example, '55' for the 55 Garfield bus)\n
    Data returned:\n
    Columns are added to the vehicles dataframe indicating the start and end time
    and the start and end distances along a pattern for each interval where a bus
    passed a stop (start_time, end_time, start_pdist, end_pdist). The estimated time 
    each bus actually arrived at the stop (est_stop_time) is also added.\n
    The dataframe returned covers all buses at all stops on the specified route'''
 
    # set up a dataframe to contain the output data
    df_output = pd.DataFrame()

    # turn vehicle data into intervals between vehicles
    vehicle_intervals = get_vehicle_intervals(vehicles, rt)

    # get pattern data from the CTA
    df_patterns = get_patterns(vehicles, rt)

    # get all stops on this route, including all patterns
    gdf_stops = get_pattern_stops(df_patterns)

    # Consider each combination of stop and pattern
    for stpid, pid, rtdir in list(zip(gdf_stops['stpid'],gdf_stops['pid'], gdf_stops['rtdir'])):

        # get a single stop on a single pattern
        gdf_this_stop_pattern = gdf_stops.loc[(gdf_stops['stpid'] == stpid) & (gdf_stops['pid'] == pid)]
        if len(gdf_this_stop_pattern) == 0:
            continue
            
        # Find the bus stop's distance along the pattern
        pdist_this_stop = gdf_this_stop_pattern['pdist'].tolist()[0]

        # find the intervals that are on this pattern
        df_this_pattern_intervals = vehicle_intervals.loc[vehicle_intervals['pid'] == pid]
        if len(df_this_pattern_intervals) == 0:
            continue

        # Filter for intervals that start ahead of the stop location and end at or beyond the stop
        def filter_intervals(stop_dist:int, start_pdist:int, end_pdist:int):
            return (start_pdist < stop_dist) & (end_pdist >= stop_dist)
    
        # Create filter for the intervals we're working on
        interval_filter = df_this_pattern_intervals.apply(
            lambda x: filter_intervals(pdist_this_stop, x['start_pdist'], x['end_pdist']), axis=1
            )
        
        # apply the filter
        df_this_pattern_stop_intervals = df_this_pattern_intervals.loc[interval_filter]
        if len(df_this_pattern_stop_intervals) == 0:
            continue

        # Add stpid, pdist, and rtdir to the data
        df_this_pattern_stop_intervals['stpid'] = stpid
        df_this_pattern_stop_intervals['stop_pdist'] = int(pdist_this_stop)
        df_this_pattern_stop_intervals['rtdir'] = rtdir


        # Estimate time each bus passed the stop (interpolated based on data at start and
        # end of the interval)
        df_this_pattern_stop_intervals['est_stop_time'] = df_this_pattern_stop_intervals.apply(
            lambda x: interpolate_stop_time(
                pdist_this_stop, 
                x['start_time'], 
                x['end_time'], 
                x['start_pdist'], 
                x['end_pdist']), axis=1
            )

        # Add the intervals with stop times to the full output dataframe
        df_output = pd.concat([df_output, df_this_pattern_stop_intervals])

    return df_output


In [ ]:

def get_headways(rt:str, vehicles:pd.DataFrame) -> pd.DataFrame:

        '''Parameters:\n
        vehicles is a dataframe obtained using get_chn_vehicles().\n
        rt is a route id as a string (for example, '55' for the 55 Garfield bus)\n
        Data returned:\n
        Columns are added to the vehicles dataframe indicating the start and end time
        and the start and end distances along a pattern for each interval where a bus
        passed a stop (start_time, end_time, start_pdist, end_pdist), the estimated time 
        each bus actually arrived at the stop (est_stop_time), and headway between each 
        bus (est_headway).  Direction of travel (rtdir) and stop id (stpid)
        are also included.\n
        The dataframe returned covers all buses at all stops on the specified route'''

        df_output = pd.DataFrame()

        # Times buses stopped at each stop on the route
        df_stoptimes = get_stoptimes(rt, vehicles).copy()

        # consider all buses stopping at a given stop moving int he same direction
        for stpid, rtdir in set(zip(df_stoptimes['stpid'], df_stoptimes['rtdir'])):

                # filter data
                df_stop_direction = df_stoptimes.loc[(df_stoptimes['stpid'] == stpid) & (df_stoptimes['rtdir'] == rtdir)]

                # Sort chronologically
                df_stop_direction.sort_values(by='est_stop_time',ascending=True, inplace=True)

                # list stop times in chronological order
                stop_times = df_stop_direction['est_stop_time'].tolist()

                # calculate previous stop time for each line
                prev_stop_times = np.roll(stop_times,1)
                df_stop_direction['previous_stop_time'] = prev_stop_times

                # calculate headway
                df_stop_direction['est_headway'] = (
                df_stop_direction['est_stop_time'] - df_stop_direction['previous_stop_time']
                )
                
                # drop previous stop time column, no longer needed
                df_stop_direction = df_stop_direction.drop('previous_stop_time', axis=1)

                # Remove headway from the first bus in the dataset since we don't have the 
                # previous bus to compare with
                df_stop_direction['est_headway'].iloc[0] = None
                
                df_output = pd.concat([df_output, df_stop_direction])

        return df_output



In [ ]:
def get_headway_stats(headways:pd.DataFrame) -> dict:
    '''Parameters:\n
    headways is a dataframe obtained using get_headways().\n
    Data returned:\n
    Statisics on the headways are returned as a dictionary.'''
    est_headways = headways['est_headway']
    stats = {
        'mean':est_headways.mean(),
        'max':est_headways.max(),
        'min':est_headways.min(),
        '25th_percentile':est_headways.quantile(0.25), # 25th percentile
        'median':est_headways.median(), # 50th percentile
        '75th_percentile':est_headways.quantile(0.75)
    }
    return stats




In [ ]:
def get_average_wait_time(headways:pd.DataFrame) -> pd.DataFrame:
    '''Parameters:\n
    headways is a dataframe obtained using get_headways().\n
    Data returned:\n
    Average wait time (AWT) value by stop ID.'''

    # AWT = SUM(D^2)/2T, where D = the duration between arrivals and T = the timeframe duration.
    # When D=T, this simplifies to AWT = D/T
    stops = pd.DataFrame()
    stops['stpid'] = headways['stpid'].unique()
    
    AWTs = []
    mean = []
    for stop in stops['stpid']:
        stop_visits = headways[headways['stpid'] == stop]
        
        start = stop_visits['start_time'].min()
        end = stop_visits['end_time'].max()
        timeframe_duration = pd.Timedelta(end-start).seconds/60.0

        headway_minutes = stop_visits['est_headway'].dt.total_seconds()/60
        AWT = ((headway_minutes)**2).sum()/(2*timeframe_duration)
        AWTs.append(AWT)

        mean.append(headway_minutes.mean())

    stops['AWT'] = AWTs
    stops['mean_headway'] = mean
    return stops


## Try it out

In [ ]:
vehicles = get_chn_vehicles('2023-01-11')

In [ ]:
vehicles

,vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,tatripid,origtatripno,tablockid,zone,scrape_file,data_time,data_hour,data_date
3969,7973,2023-01-11 02:52:00,41.873627,-87.624107,359,2642,4,Illinois Center,37819,False,10000881,238694398,N4 -792,NaN,bus_data/2023-01-11/02:52:56.json,2023-01-11 02:52:00,2,2023-01-11
3970,7936,2023-01-11 02:52:00,41.851810,-87.623825,179,10928,4,63rd,12732,False,10000871,238694396,N4 -791,NaN,bus_data/2023-01-11/02:52:56.json,2023-01-11 02:52:00,2,2023-01-11
3971,1131,2023-01-11 02:52:00,41.721710,-87.622536,88,12428,5,69th Red Line,579,False,10003773,238664290,N95 -194,NaN,bus_data/2023-01-11/02:52:56.json,2023-01-11 02:52:00,2,2023-01-11
3972,1086,2023-01-11 02:52:00,41.758954,-87.572634,269,12428,5,69th Red Line,43184,False,10003771,238664228,N95 -192,NaN,bus_data/2023-01-11/02:52:56.json,2023-01-11 02:52:00,2,2023-01-11
3973,1074,2023-01-11 02:52:00,41.773164,-87.603761,90,12429,5,95th Red Line,7298,False,10003774,238664260,N95 -193,NaN,bus_data/2023-01-11/02:52:56.json,2023-01-11 02:52:00,2,2023-01-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4094,8365,2023-01-12 02:47:00,41.968623,-87.688197,271,16551,81,Jefferson Park Blue Line,15784,False,88348717,238703288,N81 -491,NaN,bus_data/2023-01-12/02:47:56.json,2023-01-12 02:47:00,2,2023-01-12
4095,7993,2023-01-12 02:47:00,41.736362,-87.625099,269,5572,87,Western,116,False,1082088,238695539,N87 -791,NaN,bus_data/2023-01-12/02:47:56.json,2023-01-12 02:47:00,2,2023-01-12
4096,4172,2023-01-12 02:47:00,41.750329,-87.664747,268,7005,169,UPS,17948,False,9,238664069,169 -152,NaN,bus_data/2023-01-12/02:47:56.json,2023-01-12 02:47:00,2,2023-01-12
4109,8002,2023-01-12 02:49:00,41.742971,-87.663336,179,9373,9,95th Red Line,72975,False,10001957,238785508,N9 -691,NaN,bus_data/2023-01-12/02:52:56.json,2023-01-12 02:49:00,2,2023-01-12


In [ ]:
%%capture --no-display 
# turn off warnings

# Check out the Garfield bus (55)
headways = get_headways('22',vehicles)


In [ ]:
headways.sort_values('est_headway', ascending=False)

,vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,...,data_date,end_time,end_pdist,start_time,start_pdist,stpid,stop_pdist,rtdir,est_stop_time,est_headway
495,4367,2023-01-12 00:12:00,41.888115,-87.629547,354,246,22,Howard,7663,False,...,2023-01-12,2023-01-12 00:12:00,7663,2023-01-12 00:07:00,5020,14227,5229,Northbound,2023-01-12 00:07:00,0 days 19:28:00
2906,1880,2023-01-12 01:27:00,42.018013,-87.672974,10,246,22,Howard,58905,False,...,2023-01-12,2023-01-12 01:27:00,58905,2023-01-12 01:22:00,55724,15452,58258,Northbound,2023-01-12 01:26:00,0 days 06:29:00
162433,4341,2023-01-11 22:02:00,42.016412,-87.674289,58,3932,22,Howard,57343,False,...,2023-01-11,2023-01-11 22:02:00,57343,2023-01-11 21:57:00,53024,1968,57009,Northbound,2023-01-11 22:02:00,0 days 02:45:00
164788,4337,2023-01-11 22:32:00,41.874374,-87.631039,185,3936,22,Harrison,57473,False,...,2023-01-11,2023-01-11 22:32:00,57473,2023-01-11 22:27:00,54638,1871,56618,Southbound,2023-01-11 22:30:00,0 days 02:45:00
162433,4341,2023-01-11 22:02:00,42.016412,-87.674289,58,3932,22,Howard,57343,False,...,2023-01-11,2023-01-11 22:02:00,57343,2023-01-11 21:57:00,53024,14463,55980,Northbound,2023-01-11 22:00:00,0 days 02:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4433,1719,2023-01-11 03:42:00,41.995172,-87.670288,177,3936,22,Harrison,9998,False,...,2023-01-11,2023-01-11 03:42:00,9998,2023-01-11 03:37:00,1471,1784,4304,Southbound,2023-01-11 03:39:00,NaT
4035,1780,2023-01-11 02:57:00,41.897959,-87.631287,178,3936,22,Harrison,48694,False,...,2023-01-11,2023-01-11 02:57:00,48694,2023-01-11 02:52:00,44705,1851,46510,Southbound,2023-01-11 02:54:00,NaT
4034,1882,2023-01-11 02:57:00,41.932428,-87.644838,149,3936,22,Harrison,34585,False,...,2023-01-11,2023-01-11 02:57:00,34585,2023-01-11 02:52:00,26780,1822,27432,Southbound,2023-01-11 02:52:00,NaT
4035,1780,2023-01-11 02:57:00,41.897959,-87.631287,178,3936,22,Harrison,48694,False,...,2023-01-11,2023-01-11 02:57:00,48694,2023-01-11 02:52:00,44705,1854,48102,Southbound,2023-01-11 02:56:00,NaT


In [ ]:
headway_stats = get_headway_stats(headways)
headway_stats

In [ ]:
AWTs = get_average_wait_time(headways)
AWTs

,stpid,AWT,mean_headway
0,10559,8.862094,13.000000
1,10571,10.541367,13.427184
2,10626,8.809881,14.471264
3,10520,9.074558,13.679612
4,10622,8.739921,14.482759
...,...,...,...
127,10610,9.059011,13.168224
128,10530,9.136170,13.745098
129,10607,9.005654,13.177570
130,10593,9.239223,13.186916


In [ ]:
# Visualize routes and stops for the 55 bus (no headway info included in visualization yet)
patterns = get_patterns(vehicles, '55')
route = get_pattern_linestrings(patterns)
stops = get_pattern_stops(patterns)

# m = route.explore(color='blue', tiles='CartoDB_positron')
# stops.explore(m=m, color='red')


# Get Scheduled Stops

In [ ]:
# Get scheduled stop times from the GTFS feed

df_stop_times = pd.read_csv(
    'gtfs/stop_times.txt', dtype={
        'trip_id':'str',
        'arrival_time':'str',
        'departure_time':'str',
        'stop-id':'int',
        'stop_sequence':'int',
        'stop_headsign':'str',
        'pickup_type':'int',
        'shape_dist_traveled':'int'
        }, infer_datetime_format=True
    )


In [ ]:
df_stop_times['arrival_time'] = pd.to_timedelta(df_stop_times['arrival_time'])
df_stop_times['departure_time'] = pd.to_timedelta(df_stop_times['departure_time'])

In [ ]:

# Get trips from the GTFS feed

df_trips = pd.read_csv(
    'gtfs/trips.txt', dtype={
        'route_id':'str',
        'service_id':'str',
        'trip_id':'str',
        'direction_id':'int',
        'block_id':'str',
        'shape_id':'str',
        'direction':'str',
        'wheelchair_accessible':'int',
        'schd_trip_id':'str'
        }, infer_datetime_format=True
    )


In [ ]:

# df_trips


In [ ]:

# join trip data to each stop time
df_stop_schedule = df_stop_times.merge(df_trips, on='trip_id' ,how='left')


# Convert directions so they match the format from the CTA bus tracker data
df_stop_schedule['direction'] = df_stop_schedule['direction'].apply(lambda x: x+'bound')


In [ ]:

# df_stop_schedule




# Explore late night schedules
Plan to move this section to the data exploration notebook eventually


In [ ]:
# find the min and max scheduled times for each route/service/direction combination

df_minmax_schedule_time = df_stop_schedule.groupby(['route_id','service_id','direction'])['arrival_time'].aggregate([min, max])


In [ ]:

df_minmax_schedule_time


min             max
route_id service_id direction                                 
1        65201      Northbound 0 days 05:45:00 0 days 18:54:00
                    Southbound 0 days 06:15:30 0 days 19:27:30
         65301      Northbound 0 days 05:45:00 0 days 18:54:00
                    Southbound 0 days 06:15:30 0 days 19:27:30
100      65201      Eastbound  0 days 05:20:00 0 days 19:36:30
...                                        ...             ...
X98      65212      Southbound 0 days 00:40:00 0 days 01:05:00
         65312      Southbound 0 days 00:40:00 0 days 01:05:00
Y        107001     0bound     0 days 04:46:00 0 days 23:29:00
         107006     0bound     0 days 06:16:00 0 days 23:24:00
         107009     0bound     0 days 06:16:00 0 days 23:24:00

[2225 rows x 2 columns]

In [ ]:


# Find schedules that go past midnight
df_late_schedules = df_minmax_schedule_time.loc[df_minmax_schedule_time['max'] > pd.Timedelta('24:00:00')].copy()


In [ ]:

df_late_schedules.sort_values('min')


min             max
route_id service_id direction                                 
20       65306      Westbound  0 days 00:15:00 1 days 00:32:00
                    Eastbound  0 days 00:29:00 1 days 00:31:00
62       65309      Northbound 0 days 00:35:00 1 days 00:42:48
         65209      Northbound 0 days 00:35:00 1 days 00:42:11
         65206      Northbound 0 days 00:35:00 1 days 00:41:20
...                                        ...             ...
         65302      Southbound 0 days 23:47:00 1 days 00:43:00
         65305      Southbound 0 days 23:47:00 1 days 00:43:00
         65202      Southbound 0 days 23:47:00 1 days 00:43:00
4        65305      Northbound 0 days 23:59:30 1 days 01:03:00
         65302      Northbound 0 days 23:59:30 1 days 01:03:00

[646 rows x 2 columns]

In [ ]:


# Hmm.  There are several buses that the schedule shows starting at roughly 
# midnight-1am and running until midnight-1am the next day.  

# Sort for the late night schedules that have the longest run times
df_late_schedules['total_time'] = df_late_schedules['max'] - df_late_schedules ['min']

df_late_schedules.sort_values('total_time',ascending=False).head(50)


min             max      total_time
route_id service_id direction                                                 
20       65306      Westbound  0 days 00:15:00 1 days 00:32:00 1 days 00:17:00
62       65309      Northbound 0 days 00:35:00 1 days 00:42:48 1 days 00:07:48
         65209      Northbound 0 days 00:35:00 1 days 00:42:11 1 days 00:07:11
         65306      Northbound 0 days 00:35:00 1 days 00:41:54 1 days 00:06:54
         65206      Northbound 0 days 00:35:00 1 days 00:41:20 1 days 00:06:20
20       65306      Eastbound  0 days 00:29:00 1 days 00:31:00 1 days 00:02:00
62       65206      Southbound 0 days 01:32:00 1 days 00:43:00 0 days 23:11:00
         65306      Southbound 0 days 01:32:00 1 days 00:43:00 0 days 23:11:00
         65309      Southbound 0 days 01:32:00 1 days 00:35:00 0 days 23:03:00
         65209      Southbound 0 days 01:32:00 1 days 00:34:00 0 days 23:02:00
Blue     107006     Southbound 0 days 03:00:00 1 days 01:03:00 0 days 22:03:00
         107001     Northbound 0 days 03:00:00 1 days 01:03:00 0 days 22:03:00
49       65306      Northbound 0 days 03:00:00 1 days 01:01:00 0 days 22:01:00
         65206      Northbound 0 days 03:00:00 1 days 01:01:00 0 days 22:01:00
Blue     107009     Northbound 0 days 03:00:00 1 days 00:58:00 0 days 21:58:00
         107006     Northbound 0 days 03:00:00 1 days 00:58:00 0 days 21:58:00
         107009     Southbound 0 days 03:00:00 1 days 00:58:00 0 days 21:58:00
22       65306      Southbound 0 days 03:05:00 1 days 01:00:30 0 days 21:55:30
         65206      Southbound 0 days 03:05:00 1 days 01:00:30 0 days 21:55:30
         65306      Northbound 0 days 03:00:30 1 days 00:54:00 0 days 21:53:30
         65206      Northbound 0 days 03:00:30 1 days 00:54:00 0 days 21:53:30
Red      107006     Southbound 0 days 03:00:00 1 days 00:53:00 0 days 21:53:00
         107001     Southbound 0 days 03:00:00 1 days 00:53:00 0 days 21:53:00
         107009     Southbound 0 days 03:00:00 1 days 00:53:00 0 days 21:53:00
Blue     107001     Southbound 0 days 03:00:00 1 days 00:53:00 0 days 21:53:00
79       65301      Eastbound  0 days 03:00:00 1 days 00:49:30 0 days 21:49:30
         65309      Eastbound  0 days 03:00:00 1 days 00:49:30 0 days 21:49:30
         65209      Eastbound  0 days 03:00:00 1 days 00:49:30 0 days 21:49:30
         65306      Eastbound  0 days 03:00:00 1 days 00:49:00 0 days 21:49:00
         65206      Eastbound  0 days 03:00:00 1 days 00:49:00 0 days 21:49:00
49       65206      Southbound 0 days 03:12:00 1 days 01:00:00 0 days 21:48:00
         65306      Southbound 0 days 03:12:00 1 days 01:00:00 0 days 21:48:00
4        65301      Southbound 0 days 03:10:00 1 days 00:57:30 0 days 21:47:30
9        65306      Northbound 0 days 03:05:00 1 days 00:52:00 0 days 21:47:00
         65206      Northbound 0 days 03:05:00 1 days 00:52:00 0 days 21:47:00
         65309      Northbound 0 days 03:04:30 1 days 00:51:00 0 days 21:46:30
         65209      Northbound 0 days 03:04:30 1 days 00:51:00 0 days 21:46:30
4        65306      Southbound 0 days 03:10:00 1 days 00:55:30 0 days 21:45:30
         65309      Southbound 0 days 03:10:00 1 days 00:55:30 0 days 21:45:30
Red      107006     Northbound 0 days 03:05:00 1 days 00:50:00 0 days 21:45:00
         107009     Northbound 0 days 03:05:00 1 days 00:50:00 0 days 21:45:00
49       65201      Southbound 0 days 03:12:00 1 days 00:57:00 0 days 21:45:00
Red      107001     Northbound 0 days 03:05:00 1 days 00:50:00 0 days 21:45:00
49       65301      Southbound 0 days 03:12:00 1 days 00:56:30 0 days 21:44:30
79       65201      Eastbound  0 days 03:00:00 1 days 00:44:00 0 days 21:44:00
49       65209      Northbound 0 days 03:00:00 1 days 00:43:30 0 days 21:43:30
                    Southbound 0 days 03:12:00 1 days 00:55:30 0 days 21:43:30
         65309      Northbound 0 days 03:00:00 1 days 00:43:30 0 days 21:43:30
                    Southbound 0 days 03:12:00 1 days 00:55:30 0 days 21:43:30
4        65209

In [ ]:

# Looks like between 1:30 and 3am not much is happening outside of the
# 24-hour (or nearly 24-hours) schedules. perhaps 2:30 is a 
# safe cutoff time. to begin and end service days system-wide.  
# Before the first buses start.  Long enough after the 
# previous day's schedule ended to allow all remining buses to get to their final 
# stops. 

In [ ]:


# Sort by earliest start times:  2:30 still works for everything other
# than near-24 hour buses.

df_late_schedules.sort_values('min',ascending=True).head(50)



min             max      total_time
route_id service_id direction                                                 
20       65306      Westbound  0 days 00:15:00 1 days 00:32:00 1 days 00:17:00
                    Eastbound  0 days 00:29:00 1 days 00:31:00 1 days 00:02:00
62       65309      Northbound 0 days 00:35:00 1 days 00:42:48 1 days 00:07:48
         65209      Northbound 0 days 00:35:00 1 days 00:42:11 1 days 00:07:11
         65206      Northbound 0 days 00:35:00 1 days 00:41:20 1 days 00:06:20
         65306      Northbound 0 days 00:35:00 1 days 00:41:54 1 days 00:06:54
         65309      Southbound 0 days 01:32:00 1 days 00:35:00 0 days 23:03:00
         65206      Southbound 0 days 01:32:00 1 days 00:43:00 0 days 23:11:00
         65306      Southbound 0 days 01:32:00 1 days 00:43:00 0 days 23:11:00
         65209      Southbound 0 days 01:32:00 1 days 00:34:00 0 days 23:02:00
63       65309      Eastbound  0 days 03:00:00 1 days 00:23:00 0 days 21:23:00
81       65206      Westbound  0 days 03:00:00 1 days 00:18:30 0 days 21:18:30
63       65201      Eastbound  0 days 03:00:00 1 days 00:07:00 0 days 21:07:00
79       65201      Eastbound  0 days 03:00:00 1 days 00:44:00 0 days 21:44:00
         65206      Eastbound  0 days 03:00:00 1 days 00:49:00 0 days 21:49:00
87       65309      Eastbound  0 days 03:00:00 1 days 00:33:30 0 days 21:33:30
79       65209      Eastbound  0 days 03:00:00 1 days 00:49:30 0 days 21:49:30
87       65306      Eastbound  0 days 03:00:00 1 days 00:33:30 0 days 21:33:30
         65301      Eastbound  0 days 03:00:00 1 days 00:32:00 0 days 21:32:00
         65209      Eastbound  0 days 03:00:00 1 days 00:33:30 0 days 21:33:30
79       65301      Eastbound  0 days 03:00:00 1 days 00:49:30 0 days 21:49:30
87       65206      Eastbound  0 days 03:00:00 1 days 00:33:30 0 days 21:33:30
49       65206      Northbound 0 days 03:00:00 1 days 01:01:00 0 days 22:01:00
87       65201      Eastbound  0 days 03:00:00 1 days 00:34:30 0 days 21:34:30
79       65306      Eastbound  0 days 03:00:00 1 days 00:49:00 0 days 21:49:00
63       65209      Eastbound  0 days 03:00:00 1 days 00:23:00 0 days 21:23:00
49       65306      Northbound 0 days 03:00:00 1 days 01:01:00 0 days 22:01:00
63       65301      Eastbound  0 days 03:00:00 1 days 00:07:00 0 days 21:07:00
79       65309      Eastbound  0 days 03:00:00 1 days 00:49:30 0 days 21:49:30
49       65209      Northbound 0 days 03:00:00 1 days 00:43:30 0 days 21:43:30
63       65306      Eastbound  0 days 03:00:00 1 days 00:24:00 0 days 21:24:00
49       65309      Northbound 0 days 03:00:00 1 days 00:43:30 0 days 21:43:30
         65301      Northbound 0 days 03:00:00 1 days 00:14:00 0 days 21:14:00
81       65306      Westbound  0 days 03:00:00 1 days 00:18:30 0 days 21:18:30
63       65206      Eastbound  0 days 03:00:00 1 days 00:24:00 0 days 21:24:00
81       65201      Westbound  0 days 03:00:00 1 days 00:08:00 0 days 21:08:00
Blue     107006     Northbound 0 days 03:00:00 1 days 00:58:00 0 days 21:58:00
Red      107006     Southbound 0 days 03:00:00 1 days 00:53:00 0 days 21:53:00
         107001     Southbound 0 days 03:00:00 1 days 00:53:00 0 days 21:53:00
Blue     107009     Southbound 0 days 03:00:00 1 days 00:58:00 0 days 21:58:00
                    Northbound 0 days 03:00:00 1 days 00:58:00 0 days 21:58:00
         107006     Southbound 0 days 03:00:00 1 days 01:03:00 0 days 22:03:00
         107001     Southbound 0 days 03:00:00 1 days 00:53:00 0 days 21:53:00
                    Northbound 0 days 03:00:00 1 days 01:03:00 0 days 22:03:00
Red      107009     Southbound 0 days 03:00:00 1 days 00:53:00 0 days 21:53:00
49       65201      Northbound 0 days 03:00:00 1 days 00:17:00 0 days 21:17:00
22       65206      Northbound 0 days 03:00:30 1 days 00:54:00 0 days 21:53:30
         65201      Northbound 0 days 03:00:30 1 days 00:23:00 0 days 21:22:30
         65306      Northbound 0 days 03:00:30 1 days 00:54:00 0 days 21:53:30
         65301

In [ ]:

# Sort by latest end times:  Still looks OK. 
# No schedules end too far papst 1am. 
# 2:30 am should work as a cutoff
# for service days system-wide!

df_late_schedules.sort_values('max',ascending=False).head(50)


min             max      total_time
route_id service_id direction                                                 
3        65306      Southbound 0 days 05:32:42 1 days 01:07:00 0 days 19:34:18
         65206      Southbound 0 days 05:32:42 1 days 01:07:00 0 days 19:34:18
Brn      107006     0bound     0 days 04:00:00 1 days 01:05:00 0 days 21:05:00
Blue     107001     Northbound 0 days 03:00:00 1 days 01:03:00 0 days 22:03:00
8        65306      Southbound 0 days 04:05:00 1 days 01:03:00 0 days 20:58:00
Blue     107006     Southbound 0 days 03:00:00 1 days 01:03:00 0 days 22:03:00
8        65206      Southbound 0 days 04:05:00 1 days 01:03:00 0 days 20:58:00
4        65302      Northbound 0 days 23:59:30 1 days 01:03:00 0 days 01:03:30
Blue     1070127    Southbound 0 days 16:52:30 1 days 01:03:00 0 days 08:10:30
4        65305      Northbound 0 days 23:59:30 1 days 01:03:00 0 days 01:03:30
9        65301      Southbound 0 days 03:21:30 1 days 01:02:30 0 days 21:41:00
Brn      107009     0bound     0 days 05:00:00 1 days 01:02:00 0 days 20:02:00
         107001     0bound     0 days 04:00:00 1 days 01:02:00 0 days 21:02:00
49       65206      Northbound 0 days 03:00:00 1 days 01:01:00 0 days 22:01:00
8        65301      Southbound 0 days 04:05:00 1 days 01:01:00 0 days 20:56:00
         65201      Southbound 0 days 04:05:00 1 days 01:01:00 0 days 20:56:00
49       65306      Northbound 0 days 03:00:00 1 days 01:01:00 0 days 22:01:00
22       65306      Southbound 0 days 03:05:00 1 days 01:00:30 0 days 21:55:30
         65206      Southbound 0 days 03:05:00 1 days 01:00:30 0 days 21:55:30
49       65206      Southbound 0 days 03:12:00 1 days 01:00:00 0 days 21:48:00
3        65209      Southbound 0 days 07:09:42 1 days 01:00:00 0 days 17:50:18
49       65306      Southbound 0 days 03:12:00 1 days 01:00:00 0 days 21:48:00
3        65309      Southbound 0 days 07:09:42 1 days 01:00:00 0 days 17:50:18
9        65206      Southbound 0 days 03:20:30 1 days 00:59:30 0 days 21:39:00
         65306      Southbound 0 days 03:20:30 1 days 00:59:30 0 days 21:39:00
Blue     107009     Southbound 0 days 03:00:00 1 days 00:58:00 0 days 21:58:00
                    Northbound 0 days 03:00:00 1 days 00:58:00 0 days 21:58:00
151      65201      Northbound 0 days 05:00:00 1 days 00:58:00 0 days 19:58:00
Blue     107006     Northbound 0 days 03:00:00 1 days 00:58:00 0 days 21:58:00
4        65301      Southbound 0 days 03:10:00 1 days 00:57:30 0 days 21:47:30
151      65306      Northbound 0 days 05:27:30 1 days 00:57:00 0 days 19:29:30
         65206      Northbound 0 days 05:27:30 1 days 00:57:00 0 days 19:29:30
49       65201      Southbound 0 days 03:12:00 1 days 00:57:00 0 days 21:45:00
8        65206      Northbound 0 days 03:43:30 1 days 00:56:30 0 days 21:13:00
49       65301      Southbound 0 days 03:12:00 1 days 00:56:30 0 days 21:44:30
8        65306      Northbound 0 days 03:43:30 1 days 00:56:30 0 days 21:13:00
49       65309      Southbound 0 days 03:12:00 1 days 00:55:30 0 days 21:43:30
9        65309      Southbound 0 days 03:22:30 1 days 00:55:30 0 days 21:33:00
4        65309      Southbound 0 days 03:10:00 1 days 00:55:30 0 days 21:45:30
49       65209      Southbound 0 days 03:12:00 1 days 00:55:30 0 days 21:43:30
4        65306      Southbound 0 days 03:10:00 1 days 00:55:30 0 days 21:45:30
9        65209      Southbound 0 days 03:22:30 1 days 00:55:30 0 days 21:33:00
         65201      Southbound 0 days 03:21:30 1 days 00:55:30 0 days 21:34:00
82       65309      Southbound 0 days 04:14:30 1 days 00:54:30 0 days 20:40:00
         65209      Southbound 0 days 04:14:30 1 days 00:54:30 0 days 20:40:00
8        65209      Southbound 0 days 05:00:00 1 days 00:54:30 0 days 19:54:30
         65309      Southbound 0 days 05:00:00 1 days 00:54:30 0 days 19:54:30
151      65301      Northbound 0 days 05:00:00 1 days 00:54:00 0 days 19:54:00
22       65306      Northbound 0 days 03:00:30 1 days 00:54:00 0 days 21:53:30
         65206